In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import os
import re
from scipy.constants import physical_constants
import sys
sys.path.append('../..')
from MicromagneticAnalysisTools import Plot, Read, Calculate
from Utils.QuantityReader import read_quantity

matplotlib.rcParams['text.usetex'] = True

simulation_dir = '../../Micromagnetics/SimulationsAndAnalysis/MicromagneticsDataSupplied/FiniteTemperature/E00.20omega1.18/'
paramsfile = '../../Micromagnetics/SimulationsAndAnalysis/MicromagneticParams'  # File in which micromagnetic simulation parameters are stored

E0, omega = re.findall(r'[0-9]+\.[0-9]+', simulation_dir)
period = 2*np.pi / float(omega)

We first plot the "raw" data, showing the evolution of the collective coordinates over a few periods. As the starting time, we take the first integer number of periods after $t = 500$.

In [ ]:
start_time = period * np.ceil(500/period)
end_time = start_time + 2*period
start_idx = int(np.round(10*start_time))  # The magnetization texture is output every 0.1 in dimensionless time
end_idx = int(np.round(10*end_time))

temperatures = sorted([directory for directory in os.listdir(simulation_dir)], key=int)
temperatures = temperatures[:5]

fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 5), sharex=True)

cmap = matplotlib.cm.get_cmap('Reds_r')
norm = matplotlib.colors.Normalize(vmin=0, vmax=30)

x_ticks = [i * period / 2 for i in range(5)]
x_tick_labels = [r'$0$', r'$T/2$', r'$T$', r'$3T/2$', r'$2T$']
ax1.set_xticks(x_ticks)
ax1.set_xticklabels(x_tick_labels)

for T in temperatures:
    
    radii = np.load(simulation_dir + T + '/AnalysisResults/Radii.npy')
    helicities = np.load(simulation_dir + T + '/AnalysisResults/Helicities.npy')
    times = np.load(simulation_dir + T + '/AnalysisResults/times.npy')
    ax1.plot(times[start_idx:end_idx] - times[start_idx], radii[start_idx:end_idx], label=r'$\theta=' + T + '$K', color=cmap(norm(float(T))))
    ax2.plot(times[start_idx:end_idx] - times[start_idx], helicities[start_idx:end_idx] / np.pi, color=cmap(norm(float(T))))
    
ax1.legend(loc='right')
ax1.set_ylabel(r'$R$', fontsize=22)

ax2.set_xlabel(r'$t - t_0$', fontsize=22)
ax2.set_ylabel(r'$\eta / \pi$', fontsize=22)

ax1.tick_params(labelsize=18)
ax2.tick_params(labelsize=18)

plt.savefig('TemperaturesDependence.pdf', format='pdf', bbox_inches='tight')

Here, we plot the texture over one period of the evolution at a temperature of $T = 20\mathrm{K}$.

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = GridSpec(2, 2, figure=fig)

T = 20

Bz = 1.
Bz_array = np.full_like(times, Bz)
Ez_array = float(E0) * np.cos(float(omega)*times)

time_texts = {0: r'$t = t_0$', 1: r'$t = t_0 + T/4$', 2: r'$t = t_0 + T/2$', 3: r'$t = t_0 + 3T / 4$'}

# Get skyrmion's centre to correctly position the plot
dx, dy, dz = Read.sampleDiscretisation(simulation_dir + '/' + str(T) + '/Data')
initial_file = 'm%06d.ovf' %np.argmin(np.abs(times - start_time))
initial_com = Calculate.skyrmionCOM(simulation_dir + '/' + str(T) + '/Data', initial_file, dx, dy)

for i in range(4):

    period = 2*np.pi / float(omega)
    time = start_time + i * period / 4

    time_idx = np.argmin(np.abs(times - time))

    micromagnetics_ax = fig.add_subplot(gs[i//2, i%2])

    Ms = read_quantity('Ms', paramsfile)
    I1 = read_quantity('I1', paramsfile)
    I2 = read_quantity('I2', paramsfile)
    gamma = physical_constants[u'electron gyromag. ratio'][0]
    time_multiple = Ms * I2 / (gamma*I1*I1)
    field_multiple = I1**2 / (Ms * I2)
    length_multiple = np.sqrt(I2 / I1)

    plotter = Plot.MagnetizationPlotter('magnetization', simulation_dir + '/' + str(T) + '/Data', 'm%06d.ovf' %time_idx, ax=micromagnetics_ax, length_units=length_multiple, interpolation='bicubic', 
                                            plot_quiver=True, step=2, quiver_colour=[1, 1, 1], quiver_scale=3, quiver_headwidth=10, quiver_headlength=20, quiver_headaxislength=20)

    colour_plot, quiver_plot = plotter.plot()

    plot_extent = colour_plot.get_extent()
    centre_X = initial_com[0] * 1e-9 / length_multiple
    centre_Y = initial_com[1] * 1e-9 / length_multiple
    
    plot_xticks = micromagnetics_ax.get_xticks()
    plot_yticks = micromagnetics_ax.get_yticks()

    colour_plot.set_extent([plot_extent[0] - centre_X, plot_extent[1] - centre_X, plot_extent[2] - centre_Y, plot_extent[3] - centre_Y])
    quiver_plot.XY[:, 0] = quiver_plot.XY[:, 0] - centre_X
    quiver_plot.XY[:, 1] = quiver_plot.XY[:, 1] - centre_Y

    axis_ticks = [i for i in range(-4, 5, 2)]

    micromagnetics_ax.set_xticks(axis_ticks)
    micromagnetics_ax.set_yticks(axis_ticks)

    micromagnetics_ax.set_xlim(np.min(axis_ticks), np.max(axis_ticks))
    micromagnetics_ax.set_ylim(np.min(axis_ticks), np.max(axis_ticks))

    micromagnetics_ax.set_xlabel(r'$x$', fontsize=22)
    micromagnetics_ax.set_ylabel(r'$y$', fontsize=22)
    
    micromagnetics_ax.tick_params(labelsize=18)

    micromagnetics_ax.set_title(time_texts[i], size=28, ha='center', pad=20)

plt.tight_layout(pad=5)
plt.savefig('TextureFiniteTemperature.pdf', format='pdf', bbox_inches='tight')